In [ ]:
import triangle
import numpy as np
from pprint import pprint
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from math import ceil, sqrt
from scipy.sparse import csr_matrix
from scipy.integrate import dblquad

import finis

%matplotlib notebook

In [ ]:
%matplotlib notebook

### Triangulation for Testing

In [ ]:
g = {
    'vertices': np.copy(np.array([[0, 3, 3, 0], [0, 0, 1.5, 1.5]]).T),
    'segments': np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
}
maxArea = 3
mesh = triangle.triangulate(g,"pDa"+str(maxArea))
integ = finis.build_integ(mesh, order=6)
plt.figure()
finis.plot_mesh(mesh, vertex_numbers=True, triangle_numbers=True)
plt.axis("equal")
plt.show()

In [ ]:
g['vertices']

### Question 1 & 2

In [ ]:
%%latex
\begin{align}
\mathbb{P}_0: &&  \phi_0(x,y) &\equiv 1 \\[2ex]
%
\mathbb{P}_1:   &&  \phi_0(x,y) &= 1 - x - y \\
                &&  \phi_1(x,y) &= x \\
                &&  \phi_2(x,y) &= y
\end{align}

In [ ]:
def lagrange_1d(x_in, order=1):
    N = x_in.size
    
    x = x_in
    
    if len(x_in.shape) == 1:
        x = x_in[:, None]
    
    if order == 0:
        return np.ones((N, 1)), np.zeros((N, 1))
    
    if order == 1:
        phi = np.hstack((1.0-x, x))
        dxphi = np.hstack((-np.ones((N, 1)), np.ones((N, 1))))
        
        assert(phi.flags['OWNDATA'])
        assert(dxphi.flags['OWNDATA'])
        
        return  phi, dxphi
        
    raise NotImplementedError("Only orders 0 and 1 are implemented")

    
def lagrange_2d(x_in, y_in, order=1):
    N = x_in.size
    
    x = x_in
    y = y_in
    
    if len(x_in.shape) == 1:
        x = x_in[:, None]
    if len(y_in.shape) == 1:
        y = y_in[:, None]
    
    if order == 0:
        return np.ones((N, 1)), np.zeros((N, 1)), np.zeros((N, 1))
    
    if order == 1:
        phi = np.hstack((1.0-x-y, x, y))
        dxphi = np.hstack((-np.ones((N, 1)), np.ones((N, 1)), np.zeros((N, 1))))
        dyphi = np.hstack((-np.ones((N, 1)), np.zeros((N, 1)), np.ones((N, 1))))
        
        assert(phi.flags['OWNDATA'])
        assert(dxphi.flags['OWNDATA'])
        assert(dyphi.flags['OWNDATA'])
        
        return  phi, dxphi, dyphi
    
    if order == 2:
        s = b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\xc0\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x08\xc0\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xf0\xbf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x10\xc0\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x10\xc0\x00\x00\x00\x00\x00\x00\x10\xc0\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x10\xc0\x00\x00\x00\x00\x00\x00\x00\x80'
        C = np.frombuffer(s).reshape((6,6))

        V = np.hstack((np.ones_like(x), x, y, x**2, x*y, y**2))
        dxV = np.hstack((np.zeros_like(x), np.ones_like(x), np.zeros_like(x), 2*x, y, np.zeros_like(x)))
        dyV = np.hstack((np.zeros_like(x), np.zeros_like(x), np.ones_like(x), np.zeros_like(x), x, 2*y))
        
        phi = np.dot(V, C)
        dxphi = np.dot(dxV, C)
        dyphi = np.dot(dyV, C)
        
        assert(phi.flags['OWNDATA'])
        assert(dxphi.flags['OWNDATA'])
        assert(dyphi.flags['OWNDATA'])
        
        return  phi, dxphi, dyphi
        
    raise NotImplementedError("Only orders 0, ..., 2 are implemented")

### Question 3 & 4

In [ ]:
def fe_dof(mesh, order_shape=1):
    """
    Returns
    -------
    dof : ndarray (N_dof, 2)
        dif[i,:] = (x, y) of DOF number i
    num_dof : ndarray (N_tri, Nb)
        num_dof[i,:] the numbers of the DOF for triangle i
    
    """
    if (order_shape < 0) or (order_shape > 2):
        raise NotImplementedError("Only shape functions of order 0, 1, 2 are implemented")
    
    N_tri = mesh['triangles'].shape[0] # Number of triangles
    N_vert = mesh['vertices'].shape[0] # number of vertices
    
    if order_shape == 0:
        # Center of gravity
        dof = finis.triangle_cog(mesh)
        num_dof = np.arange(N_tri)[:,None]
        
    if order_shape == 1:
        # Vertices
        dof = np.copy(mesh['vertices'])
        num_dof = np.copy(mesh['triangles'])
    
    if order_shape == 2:
        # Vertices and middle of edges
        from edges import meshEdges
        edge, edge_markers, ElementEdges = meshEdges(mesh)

        xy_edge1 = mesh['vertices'][edge[:,0], :] # start of edges
        xy_edge2 = mesh['vertices'][edge[:,1], :] # end of edges
        xy_edge = 0.5*(xy_edge1 + xy_edge2) # middle point
        
        dof = np.vstack((mesh['vertices'], xy_edge))
        
        num_dof = np.zeros((N_tri, 6), dtype=mesh['triangles'].dtype)
        num_dof[:, 0:3] = mesh['triangles']
        num_dof[:, 3:6] = ElementEdges + N_vert
    
    return dof, num_dof
        
def fe_space(mesh, order_shape=1, order_int=2, return_integ=False, return_h=False):
    """fe = finite element"""
    
    # tables for finite element dofs
    dof, num_dof = fe_dof(mesh, order_shape=order_shape)
    N_dof = dof.shape[0] # total number of dof
    N_b = num_dof.shape[1] # number of shape functions per triangle
    
    N_tri = mesh['triangles'].shape[0] # Number of triangles
    
    xh, yh, wh = finis.form_int_2d(order=order_int)
    N_int = xh.size # number integration points per triangle

    phi, dxphi, dyphi = lagrange_2d(xh, yh, order=order_shape)
    assert phi.shape[0]==N_int, "Number of integration points"
    assert phi.shape[1]==N_b, "Number of shape functions per triangle" 

    
    # Attention: The following code is fully vectorized and horrible to understand. Sorry

    ### U ###
    rows = np.repeat(np.arange(N_tri*N_int), N_b) # 0,0,...0, 1, 1, ...., 1, ....
    cols = np.tile(num_dof, reps=(1,N_int)).flatten(order='C') # dofs of triangle 0, dofs of traingle 0 , ...., dofs of triangle 1, ...
    data = np.tile(phi.flatten(order='C'), N_tri)

    U = csr_matrix((data, (rows, cols)), shape=(N_tri*N_int, N_dof))


    ### Derivative Transforms ###
    xT = finis.triangle_x(mesh)
    yT = finis.triangle_y(mesh)

    dx_dxh = xT[:,1]-xT[:,0]
    dx_dyh = xT[:,2]-xT[:,0]
    dy_dxh = yT[:,1]-yT[:,0]
    dy_dyh = yT[:,2]-yT[:,0]

    det = dx_dxh*dy_dyh - dx_dyh*dy_dxh
    det_inv = 1.0 / det
    assert np.amax(np.abs(det_inv)) < 1e5, "Badly scaled triangles"

    dxh_dx = det_inv * dy_dyh
    dyh_dy = det_inv * dx_dxh
    dxh_dy = - det_inv * dx_dyh
    dyh_dx = - det_inv * dy_dxh


    ### Transform Differentials ###
    dphi_dxh = np.tile(dxphi.flatten(order='C'), N_tri)
    dphi_dyh = np.tile(dyphi.flatten(order='C'), N_tri)

    dphi_dx = dphi_dxh * np.repeat(dxh_dx, N_b*N_int) + dphi_dyh * np.repeat(dyh_dx, N_b*N_int)
    dphi_dy = dphi_dxh * np.repeat(dxh_dy, N_b*N_int) + dphi_dyh * np.repeat(dyh_dy, N_b*N_int)


    ### DUX, DUY ###
    DUX = csr_matrix((dphi_dx, (rows, cols)), shape=(N_tri*N_int, N_dof))
    DUY = csr_matrix((dphi_dy, (rows, cols)), shape=(N_tri*N_int, N_dof))
    
    fe = {
        'dof': dof,
        'num_dof': num_dof,
        'U': U,
        'DUX': DUX,
        'DUY': DUY,
        'w': np.repeat(np.abs(det), N_int) * np.tile(wh, N_tri),
    }
    
    if return_integ:
        x_int = np.zeros((N_int * N_tri, ))
        y_int = np.zeros((N_int * N_tri, ))
    
        for i in range(N_tri):
            x_int[N_int*i:N_int*(i+1)] = xT[i,0] + dx_dxh[i]*xh + dx_dyh[i]*yh
            y_int[N_int*i:N_int*(i+1)] = yT[i,0] + dy_dxh[i]*xh + dy_dyh[i]*yh
        
        fe['integ'] = np.hstack((x_int[:, None], y_int[:, None]))
        
    if return_h:
        from edges import meshEdges
        edge, edge_markers, ElementEdges = meshEdges(mesh)

        xy_edge1 = mesh['vertices'][edge[:,0], :] # start of edges
        xy_edge2 = mesh['vertices'][edge[:,1], :] # end of edges
        
        fe['h'] = np.sqrt(np.sum((xy_edge1-xy_edge2)**2, axis=1))

    return fe

In [ ]:
order_shape = 2
fe = fe_space(mesh,
              order_shape=order_shape,
              order_int=2,
              return_integ=True)

plt.figure()
finis.plot_mesh(mesh, triangle_numbers=True)
plt.plot(fe['dof'][:,0], fe['dof'][:,1], 'ko', label="DOF, k = {}".format(order_shape))
plt.plot(fe['integ'][:,0], fe['integ'][:,1], '+r', label='integ')

for i in range(fe['integ'].shape[0]):
    plt.text(fe['integ'][i,0], fe['integ'][i,1], str(i), color='r')
for i in range(fe['dof'].shape[0]):
    plt.text(fe['dof'][i,0], fe['dof'][i,1], str(i), color='r')
    
plt.legend()
plt.axis("equal")
plt.show()

### Question 6

In [ ]:
g = {
    'vertices': np.copy(np.array([[0, 1, 1, 0], [0, 0, 1., 1.]]).T),
    'segments': np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
}
maxArea = 0.0001
mesh = triangle.triangulate(g,"qpDa"+str(maxArea))

print("number of vertices = {}".format(mesh['vertices'].shape[0]))

In [ ]:
%time fe = fe_space(mesh, order_shape=2, order_int=2)

plt.figure()
finis.plot_mesh(mesh, vertex_numbers=True, triangle_numbers=True)
plt.plot(fe['dof'][:,0], fe['dof'][:,1], 'ko', label="DOF, k = {}".format(order_shape))
plt.legend()
plt.axis("equal")
plt.show()

In [ ]:
fun = lambda x, y: np.exp(x)
dx_fun = lambda x, y: np.exp(x)
dy_fun = lambda x, y: np.zeros_like(x)

%time fe = fe_space(mesh, order_shape=2, order_int=4, return_integ=True)

u_h = fun(fe['dof'][:,0], fe['dof'][:,1])
u_int = fe['U'].dot(u_h)
dx_u_int = fe['DUX'].dot(u_h)
dy_u_int = fe['DUY'].dot(u_h)


u_int_control = fun(fe['integ'][:,0], fe['integ'][:,1])
dx_u_int_control = dx_fun(fe['integ'][:,0], fe['integ'][:,1])
dy_u_int_control = dy_fun(fe['integ'][:,0], fe['integ'][:,1])


print("Max err. at x_int (eval) = {}".format(np.amax(np.abs(u_int - u_int_control))))
print("Max err. at x_int (d/dx) = {}".format(np.amax(np.abs(dx_u_int - dx_u_int_control))))
print("Max err. at x_int (d/dy) = {}".format(np.amax(np.abs(dy_u_int - dy_u_int_control))))

print("\nNumeric Integral  = {}".format(np.sum(u_int*fe['w'])))
print("Analytic Integral = {}".format(np.expm1(1)))

print("Number of vertices   = {}".format(u_h.size))
print("Number of int points = {}".format(u_int.size))

In [ ]:
fun_f = lambda x, y: np.exp(x+y)
fun_g = lambda x, y: np.sin(x*(1-x)*y**5)


%time fe = fe_space(mesh, order_shape=2, order_int=2, return_integ=False)

f_h = fun_f(fe['dof'][:,0], fe['dof'][:,1])
g_h = fun_g(fe['dof'][:,0], fe['dof'][:,1])

f_int = fe['U'].dot(f_h)
g_int = fe['U'].dot(g_h)

dx_f_int = fe['DUX'].dot(f_h)
dx_g_int = fe['DUX'].dot(g_h)

I1 = np.sum(fe['w'] * f_int * dx_g_int)
I2 = np.sum(fe['w'] * g_int * dx_f_int)

print(I1+I2)

### Convergence Analysis: $\mathbb{L}^2$

In [ ]:
maxArea = np.logspace(0, 4, 20, base=0.1)
L2_error = np.zeros(maxArea.shape)
hs = np.zeros(maxArea.shape)
fun = lambda x, y: np.exp(x)
I_ana = np.expm1(1)

for i in range(maxArea.size):
    mesh = triangle.triangulate(g,"qpDa"+str(maxArea[i]))
    fe = fe_space(mesh, order_shape=2, order_int=6, return_h=True, return_integ=True)
    u_h = fun(fe['dof'][:,0], fe['dof'][:,1])
    u_int = fe['U'].dot(u_h)
    f_int = fun(fe['integ'][:,0], fe['integ'][:,1])

    
    L2_error[i] = np.sqrt(np.sum(fe['w'] * (u_int-f_int)**2))
    hs[i] = np.mean(fe['h'])

plt.figure()
plt.loglog(hs, L2_error, '+-')
plt.loglog(np.array([hs[0], hs[-1]]), np.array([L2_error[0], L2_error[-1]]), 'r:')
plt.show()

In [ ]:
i = -6
(np.log(L2_error[i])-np.log(L2_error[-1])) / (np.log(hs[i])-np.log(hs[-1]))

In [ ]:
maxArea = np.logspace(0, 4, 20, base=0.1)
H1_error = np.zeros(maxArea.shape)
hs = np.zeros(maxArea.shape)

fun = lambda x, y: np.exp(x)
dx_fun = lambda x, y: np.exp(x)
dy_fun = lambda x, y: np.zeros_like(x)

I_ana = np.expm1(1)

for i in range(maxArea.size):
    mesh = triangle.triangulate(g,"qpDa"+str(maxArea[i]))
    fe = fe_space(mesh, order_shape=2, order_int=4, return_h=True, return_integ=True)
    u_h = fun(fe['dof'][:,0], fe['dof'][:,1])
    u_int = fe['U'].dot(u_h)
    dx_u_int = fe['DUX'].dot(u_h)
    dy_u_int = fe['DUY'].dot(u_h)
    
    f_int = fun(fe['integ'][:,0], fe['integ'][:,1])
    dx_f_int = dx_fun(fe['integ'][:,0], fe['integ'][:,1])
    dy_f_int = dy_fun(fe['integ'][:,0], fe['integ'][:,1])

    
    H1_error[i] = \
        np.sqrt(np.sum(fe['w'] * (u_int-f_int)**2)) + \
        np.sqrt(np.sum(fe['w'] * (dx_u_int-dx_f_int)**2)) + \
        np.sqrt(np.sum(fe['w'] * (dy_u_int-dy_f_int)**2))
    hs[i] = np.mean(fe['h'])

plt.figure()
plt.loglog(hs, H1_error, '+-')
plt.loglog(np.array([hs[0], hs[-1]]), np.array([H1_error[0], H1_error[-1]]), 'r:')
plt.show()

In [ ]:
i = -6
(np.log(H1_error[i])-np.log(H1_error[-1])) / (np.log(hs[i])-np.log(hs[-1]))